In [16]:
# Step 1: Check if the TBB module exists
try:
    import tbb
    tbb_path = tbb.__file__
    print(f"TBB path: {tbb_path}")
except ImportError:
    print("TBB module is not installed.")

# Step 2: Remove the TBB directory if it exists
import shutil
import os

if 'tbb_path' in locals():
    tbb_dir = os.path.dirname(tbb_path)
    if os.path.exists(tbb_dir):
        print(f"Removing directory: {tbb_dir}")
        shutil.rmtree(tbb_dir)
    else:
        print(f"Directory does not exist: {tbb_dir}")
else:
    print("TBB module is not installed, so no directory to remove.")

# Step 3: Install PyTorch
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python get-pip.py

!pip install torch torchvision torchaudio

TBB path: C:\Users\minam\anaconda3\lib\site-packages\tbb\__init__.py
Directory does not exist: C:\Users\minam\anaconda3\lib\site-packages\tbb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2224k  100 2224k    0     0  3515k      0 --:--:-- --:--:-- --:--:-- 3519k


   ---------------------------------------- 1.8/1.8 MB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 931.1/931.1 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 65.8/65.8 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 60.9/60.9 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 159.7/159.7 MB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 1.2/1.2 MB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 2.4/2.4 MB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 228.5/228.5 MB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 3.5/3.5 MB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 286.9/286.9 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 2.6/2.6 MB 3.9 MB/s eta 0:00:00
   ---------------------------------------- 177.6/177.6 kB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 133.3/133.3 kB 4.0

In [17]:
import numpy as np
from scipy.linalg import svd
import torch

print("How many rows do you want your matrix to have? ")
rows = input()
print("How many columns do you want your matrix to have? ")
cols = input()

X = np.zeros((int(rows), int(cols)))
for i in range(int(rows)):
    for j in range(int(cols)):
        print("enter the value for row " +str(i)+ " and column " +str(j)+ " : ")
        X[i][j] = input()
        
print(X)

How many rows do you want your matrix to have? 
2
How many columns do you want your matrix to have? 
2
enter the value for row 0 and column 0 : 
1
enter the value for row 0 and column 1 : 
2
enter the value for row 1 and column 0 : 
3
enter the value for row 1 and column 1 : 
4
[[1. 2.]
 [3. 4.]]


In [18]:
#Step 1

XT = X.transpose()
XTX = np.matmul(XT,X)

In [19]:
#Step 2

eigenvals = np.linalg.eig(XTX)
singular_vals = []
for i in eigenvals[0]:
    if i >= 0:
        singular_vals.append(np.sqrt(i))
    else:
        singular_vals.append(np.sqrt(abs(i)))

In [20]:
#Step 3

singular_vals.sort(reverse = True)

count = 0
for i in singular_vals:
    count += 1
sigma = np.zeros((count, count))

for i in range(count):
    sigma[i][i] = singular_vals[i]

sigma_inv = np.linalg.inv(sigma)

In [21]:
#Step 4

eigenvalues, eigenvectors = np.linalg.eigh(XTX)
sorted_indices = np.argsort(eigenvalues)[::-1]
V = -1*eigenvectors[:, sorted_indices]
VT = V.T

In [22]:
#Step 5 (U=X*V*sigma_inv)

U = np.matmul(X, np.matmul(V, sigma_inv))
A = np.matmul(U, np.matmul(sigma, VT))

print("Original Matrix After SVD:")
print(A)
print("U:")
print(U)
print("Sigma:")
print(sigma)
print("V Transpose:")
print(VT)

Original Matrix After SVD:
[[1. 2.]
 [3. 4.]]
U:
[[-0.40455358 -0.9145143 ]
 [-0.9145143   0.40455358]]
Sigma:
[[5.4649857  0.        ]
 [0.         0.36596619]]
V Transpose:
[[-0.57604844 -0.81741556]
 [ 0.81741556 -0.57604844]]


In [23]:
def SVD(X):
    XT = X.transpose()
    XTX = np.matmul(XT,X)
    
    eigenvals = np.linalg.eig(XTX)
    singular_vals = []
    for i in eigenvals[0]:
        if i >= 0:
            singular_vals.append(np.sqrt(i))
        else:
            singular_vals.append(np.sqrt(abs(i)))
            
    singular_vals.sort(reverse = True)

    count = 0
    for i in singular_vals:
        count += 1
    sigma = np.zeros((count, count))

    for i in range(count):
        sigma[i][i] = singular_vals[i]

    sigma_inv = np.linalg.inv(sigma)
    
    eigenvalues, eigenvectors = np.linalg.eigh(XTX)
    sorted_indices = np.argsort(eigenvalues)[::-1]
    V = -1*eigenvectors[:, sorted_indices]
    VT = V.T
    
    U = np.matmul(X, np.matmul(V, sigma_inv))
    A = np.matmul(U, np.matmul(sigma, VT))
    #check answers using svd function
    
    return U, sigma, VT

In [31]:
'''
This is a program that computes the Tucker Decomposition of an 
order-3 tensor using the Higher-Order Orthogonal Iteration 
Algorithm (HOOI) and the SVD program above.
Params: A (original tensor)
Returns: L, R, V (order matrices), B (core tensor)
'''

def Tucker(A):
    R = A[0]
    V = A[1]
    RT = R.transpose()
    VT = V.T
    C = torch.mul(A, torch.mul(RT, VT))
    L = SVD(C[0])
    LT = L.T
    D = torch.mul(A, torch.mul(LT, VT))
    R = SVD(D[1])
    E = torch.mul(A, torch.mul(LT, RT))
    V = SVD(E[2])
    B = torch.mul(E, VT)
    return B

In [32]:
A = [[[1,2,3],[4,5,6],[7,8,9]],[[1,2,3],[4,5,6],[7,8,9]]]
B = Tucker(A)
print(B)

AttributeError: 'list' object has no attribute 'transpose'